In [10]:
import numpy as np
import torch
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import random
import numpy as np
import random
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import hog
from skimage import color, exposure
from sklearn.pipeline import Pipeline
import torch
import torchvision.datasets as datasets
from torch.utils.data import Subset, DataLoader
import torchvision.transforms as transforms
from numpy.random import RandomState
import torchvision
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
import random
from torchvision.models import resnet18


In [16]:
from numpy.random import RandomState
import numpy as np
import torch.optim as optim
from torch.utils.data import Subset


from torchvision import datasets, transforms
start_time = time.time()
normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))


transform_train = transforms.Compose([transforms.ToTensor(), normalize])
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                   std=[0.229, 0.224, 0.225])
# Data augmentation during training:
# transform_train = transforms.Compose([
#                                     transforms.RandomCrop(32, padding=2, padding_mode='reflect'),
#                                     transforms.RandomGrayscale(),
#                                     # transforms.Resize((224, 224)),
#                                     transforms.RandomHorizontalFlip(),
#                                     torchvision.transforms.RandomAffine(degrees=30),
#                                     transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
#                                     transforms.ToTensor(),
#                                     normalize]) #careful to keep this one same

transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

##### Cifar Data
cifar_data = datasets.CIFAR10(root='./data',train=True, transform=transform_train, download=True)

#We need two copies of this due to weird dataset api
cifar_data_val = datasets.CIFAR10(root='./data',train=True, transform=transform_val, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
start_time = time.time()
# Extract features and labels from loaders
def extract_features_and_labels(loader):
    for images, labels in loader:
        features = images.view(images.size(0), -1).numpy()
        labels = labels.numpy()
        return features, labels

# Load CIFAR-10 dataset
cifar_data = datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
random.seed(42)
svm_accs = []
knn_accs = []
# Loop over different splits of the data
for seed in range(1, 5):
    random.seed(seed)

    # Select random classes
    classes = random.sample(range(10), 2)

    # Shuffle indices for selecting samples
    indices = np.arange(len(cifar_data))
    random.shuffle(indices)
    prng = RandomState(seed)
    random_permute = prng.permutation(np.arange(0, 1000))
    classes =  prng.permutation(np.arange(0,10))

    # Create training and validation subsets
    train_indices = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:25]] for classe in classes[0:2]])
    val_indices = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[25:225]] for classe in classes[0:2]])

    # Create Subset datasets
    train_data = Subset(cifar_data, train_indices)
    val_data = Subset(cifar_data_val, val_indices)

    # Create DataLoader instances
    train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_data, batch_size=128, shuffle=False, num_workers=2)

    print('Num Samples For Training %d Num Samples For Val %d' % (len(train_data), len(val_data)))
    x_train, y_train = extract_features_and_labels(train_loader)
    x_test, y_test = extract_features_and_labels(val_loader)

    # Define a pipeline with feature extraction and SVM classifier
    svm_pipeline = Pipeline([
    ('feature_extraction', PCA(n_components=33)),
    ('clf', SVC(kernel='linear', random_state=42))
    ])


    param_grid = {
    'feature_extraction': [PCA(n_components=33), LDA(n_components=1)],
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear']
    }


    # Grid search for SVM
    svm_grid_search = GridSearchCV(svm_pipeline, param_grid, cv=3, n_jobs=1, verbose=3)
    svm_grid_search.fit(x_train, y_train)


    # Evaluate best models
    svm_best_model = svm_grid_search.best_estimator_

    svm_predictions = svm_best_model.predict(x_test)

    svm_accuracy = accuracy_score(y_test, svm_predictions)

    print("SVM Accuracy:", svm_accuracy)
    print(classification_report(y_test, svm_predictions))

    # Testing
    svm_accs.append(svm_accuracy)

# Compute and print mean and standard deviation of accuracy
svm_accs = np.array(svm_accs)
print('SVM Acc over 5 instances: %.2f +- %.2f' % (svm_accs.mean(), svm_accs.std()))

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")


Files already downloaded and verified
Num Samples For Training 50 Num Samples For Val 400


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.706 total time=   0.0s
[CV 2/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.529 total time=   0.0s
[CV 3/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.688 total time=   0.0s
[CV 1/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=LinearDiscriminantAnalysis(n_components=1);, score=0.471 total time=   0.1s
[CV 2/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=LinearDiscriminantAnalysis(n_components=1);, score=0.471 total time=   0.1s
[CV 3/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=LinearDiscriminantAnalysis(n_components=1);, score=0.688 total time=   0.1s
[CV 1/3] END clf__C=1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.706 total time=   0.0s
[CV 2/3] END clf__C=1, clf__kernel=linear, feature

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was call

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.588 total time=   0.0s
[CV 2/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.588 total time=   0.0s
[CV 3/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.500 total time=   0.0s
[CV 1/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=LinearDiscriminantAnalysis(n_components=1);, score=0.353 total time=   0.1s
[CV 2/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=LinearDiscriminantAnalysis(n_components=1);, score=0.647 total time=   0.1s
[CV 3/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=LinearDiscriminantAnalysis(n_components=1);, score=0.500 total time=   0.1s
[CV 1/3] END clf__C=1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.588 total time=   0.0s
[CV 2/3] END clf__C=1, clf__kernel=linear, feature

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was call

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.647 total time=   0.0s
[CV 2/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.647 total time=   0.0s
[CV 3/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.688 total time=   0.0s
[CV 1/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=LinearDiscriminantAnalysis(n_components=1);, score=0.471 total time=   0.1s
[CV 2/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=LinearDiscriminantAnalysis(n_components=1);, score=0.706 total time=   0.1s
[CV 3/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=LinearDiscriminantAnalysis(n_components=1);, score=0.625 total time=   0.1s
[CV 1/3] END clf__C=1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.647 total time=   0.0s
[CV 2/3] END clf__C=1, clf__kernel=linear, feature

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was call

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.588 total time=   0.0s
[CV 2/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.824 total time=   0.0s
[CV 3/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.812 total time=   0.0s
[CV 1/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=LinearDiscriminantAnalysis(n_components=1);, score=0.647 total time=   0.1s
[CV 2/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=LinearDiscriminantAnalysis(n_components=1);, score=0.824 total time=   0.1s
[CV 3/3] END clf__C=0.1, clf__kernel=linear, feature_extraction=LinearDiscriminantAnalysis(n_components=1);, score=0.750 total time=   0.1s
[CV 1/3] END clf__C=1, clf__kernel=linear, feature_extraction=PCA(n_components=33);, score=0.588 total time=   0.0s
[CV 2/3] END clf__C=1, clf__kernel=linear, feature

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
